In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password

In [35]:
zika_path = "Resources/ZikaData2015.xlsx"
df_zika_init = pd.read_excel(zika_path)
demograph_path = "Resources/PEP_2018_PEPTCOMP_with_ann.csv"
df_demo_raw_init = pd.read_csv(demograph_path)
df_demo_raw = df_demo_raw_init.iloc[1:]
df_zika_raw = df_zika_init.iloc[1:]
df_zika_raw.head()

,States,Symptomatic disease cases*(N=62),Unnamed: 2,Presumptive viremic blood donors†(N=0),Unnamed: 4
1,Alabama,0,0,0,0
2,Arizona,0,0,0,0
3,Arkansas,2,-3,0,0
4,California,13,-21,0,0
5,Colorado,0,0,0,0


In [3]:
teens_data= pd.read_csv("Teen_birthrates.csv", header=None, low_memory=False)
teens_data= teens_data.sample(frac=1).reset_index(drop=True)

In [4]:
df_demo_raw.rename (columns={'GEO.display-label': 'States'}, inplace=True)
df_demo_select = df_demo_raw[["States","births4201072018","deaths4201072018"]]

C:\Users\Admin\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [5]:
df_merged = df_demo_select.merge(df_zika_raw, on="States")

In [6]:
df_merged_drop = df_merged.drop(columns = ["Unnamed: 2", "Unnamed: 4", "Presumptive viremic blood donors†(N=0)"])

In [7]:
df_merged_renamed = df_merged_drop.rename(columns = {"States":"State","births4201072018":"Births","deaths4201072018":"Deaths","Symptomatic disease cases*(N=62)":"Zika_Cases"})
df_merged_renamed.head()

,State,Births,Deaths,Zika_Cases
0,Alabama,484329,418734,0
1,Arizona,703872,434572,0
2,Arkansas,314960,251951,2
3,California,4078895,2094459,13
4,Colorado,543471,289139,0


In [41]:
teens_data_clean = teens_data[[0, 1, 6]]
teens_data_clean = teens_data_clean.rename(columns={teens_data.columns[0]:"Year",\
                                                    teens_data.columns[1]:"State",\
                                                    teens_data.columns[6]:"Birthrate"})
teen_byear = teens_data_clean.sort_values(by=['Year'], ascending=False).reset_index(drop=True)[1:]
teen_byear['Year']=pd.to_datetime(teen_byear['Year'])
teen_byear['Year']=teen_byear[teen_byear['Year'].dt.year == 2015]
teen_birthdate = teen_byear.drop(columns=['Year'])
teen_birthdate["Birthrate"] = pd.to_numeric(teen_birthdate["Birthrate"])
df_teenmerge = teen_birthdate.groupby(["State"]).mean()
df_teenmerge

,Birthrate
State,
Alabama,48.240726
Alaska,42.806076
Arizona,54.496552
Arkansas,56.800268
California,32.381431
Colorado,35.140311
Connecticut,16.481804
Delaware,36.530293
District of Columbia,41.437791


In [17]:
df_finalmerge = df_merged_renamed.merge(teen_birthdate, on="State")
df_finalmerge.head()

,State,Births,Deaths,Zika_Cases,Birthrate
0,Alabama,484329,418734,0,37.866381
1,Alabama,484329,418734,0,41.848795
2,Alabama,484329,418734,0,15.325253
3,Alabama,484329,418734,0,51.103272
4,Alabama,484329,418734,0,27.396421


In [10]:
#Export to SQL
#Make sure to run queries.sql to set up database
#Remember to create config.py with user and password
rds_connection_string = user + ":" + password + "@127.0.0.1/state_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [11]:
engine.table_names()

['state_data']

In [23]:
df_finalmerge.to_sql(name='state_data', con=engine, if_exists='append', index=False)

IntegrityError: (_mysql_exceptions.IntegrityError) (1062, "Duplicate entry 'Alabama' for key 'PRIMARY'")
[SQL: INSERT INTO state_data (`State`, `Births`, `Deaths`, `Zika_Cases`, `Birthrate`) VALUES (%s, %s, %s, %s, %s)]
[parameters: (('Alabama', '484329', '418734', 0, '37.866381'), ('Alabama', '484329', '418734', 0, '41.848795'), ('Alabama', '484329', '418734', 0, '15.325253'), ('Alabama', '484329', '418734', 0, '51.103272'), ('Alabama', '484329', '418734', 0, '27.396421'), ('Alabama', '484329', '418734', 0, '30.815264'), ('Alabama', '484329', '418734', 0, '39.423994'), ('Alabama', '484329', '418734', 0, '24.118533')  ... displaying 10 of 40456 total bound parameter sets ...  ('Wyoming', '61495', '38474', 0, '39.079718'), ('Wyoming', '61495', '38474', 0, '54.87663'))]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [21]:
pd.read_sql_query('select * from state_data', con=engine).head()

,State,Births,Deaths,Zika_Cases,Birthrate
0,Alabama,484329,418734,0,None
1,Arizona,703872,434572,0,None
2,Arkansas,314960,251951,2,None
3,California,4078895,2094459,13,None
4,Colorado,543471,289139,0,None
